In [13]:
import stim
from rgs_theoretical_model import *
import math

In [ ]:
def binom_prob(p, n, m, select='all'):
    if select not in ['all', 'even', 'odd']:
        raise ValueError('select mode does not support given "{select}".')
    if select == 'all':
        return math.comb(n, m) * p ** m * (1 - p) ** (n - m)
    if select == 'odd' and m % 2 == 1:
        return math.comb(n, m) * p ** m * (1 - p) ** (n - m)
    if select == 'even' and m % 2 == 0:
        return math.comb(n, m) * p ** m * (1 - p) ** (n - m)
    return 0

class RgsModel:
    def __init__(
        self,
        num_hops,
        distance_between_hop,
        branching_parameters,
        loss_in_db_per_km,
        emitter_meas_error_probability,
        inner_photon_depo_error_probability,
        outer_photon_depo_error_probability,
    ):
        self.num_hops = num_hops
        self.distance = distance_between_hop
        self.bv = branching_parameters

        attenuation_distance = 10 / (np.log(10) * loss_in_db_per_km)
        self.p_ph = np.exp(-distance_between_hop / 2 / attenuation_distance)
        self.n = len(self.bv)

        print(f'n = {self.n}')

        self.meas_err_prob = emitter_meas_error_probability
        # should we make this 2/3 of the set value instead?

        # TODO: use epsilon differently
        # self.epsilon_inner = inner_photon_depo_error_probability
        # self.epsilon_outer = outer_photon_depo_error_probability
        if inner_photon_depo_error_probability != outer_photon_depo_error_probability:
            raise ValueError("Currently does not support different depolarizing probability for inner and outer photons")
        self.epsilon = inner_photon_depo_error_probability

    def exhaustive_eval_e_logical_x(self):
        # there are 3 states for each photon; lost, no_error, has_error
        

    def r(self, k):
        # probability of at least one indirect measurement to succeed at level k (k = 0 is logical)
        if k >= self.n:
            raise ValueError(f'r({k}): called with value exceeding the expected range')
            return 0
        return 1 - (1 - self.s(k)) ** self.bv[k]

    def s(self, k):
        # success probablity for a single indirect measurement at level k (k = 0 is logical)
        if k == self.n - 1:
            # second to last layer, there's only X below
            return self.p_ph
        if k >= self.n:
            # we can't perform indirect measurement anymore, so the probability of success is 0
            raise ValueError(f's({k}): called with value exceeding the expected range')
            return 0
        return self.p_ph * self.prob_mz(k + 2) ** self.bv[k + 1]

    def prob_mz(self, k):
        if k == self.n:
            return self.p_ph
        if k > self.n:
            raise ValueError(f'prob MZ was called with k > n ({k} > {self.n})')
        return self.p_ph + (1 - self.p_ph) * self.r(k)

    def prob_indirect_success_given(self, k, mk):
        return binom_prob(self.s(k), self.bv[k], mk)

    def prob_indirect_given_meas_success(self, k):
        return self.r(k) / self.prob_mz(k)

    def e_indirect(self, k) -> float:
        # print(f'e_indirect({k})')
        prob = 0
        bk = self.bv[k]
        for mk in range(1, bk + 1):
            prob += self.prob_indirect_success_given(k, mk) * self.e_indirect_given(k, mk)
        return prob / self.r(k)

    def e_indirect_given(self, k, mk):
        # print(f'e_indirect_given({k}, {mk})')
        prob = 0
        if mk % 2 == 1:
            for j in range((mk + 1) // 2, mk + 1):
                prob += math.comb(mk, j) * self.e_indirect_single(k) ** j * (1 - self.e_indirect_single(k)) ** (mk - j)
        else:
            for j in range((mk + 1) // 2, mk):
                prob += math.comb(mk - 1, j) * self.e_indirect_single(k) ** j * (1 - self.e_indirect_single(k)) ** (mk - 1 - j)
        return prob

    def e_indirect_single(self, k) -> float:
        # print(f'e_indirect_single({k})')
        if k == self.n - 1:
            return self.epsilon
        if k == self.n - 2:
            return (1 - (1 - 2*self.epsilon) ** (self.bv[k+1] + 1)) / 2
        prob = 0
        b = self.bv[k+1]
        for nk in range(b + 1):
            prob += math.comb(b, nk) * self.prob_indirect_given_meas_success(k+2) ** (b - nk) * (1 - self.prob_indirect_given_meas_success(k+2)) ** nk * self.e_direct(nk, k)
        return prob

    def e_direct(self, n, k):
        # print(f'e_indirect({n}, {k})')
        prob = 0
        for i in range(n + 2):
            part1 = math.comb(n + 1, i) * self.epsilon ** i * (1 - self.epsilon) ** (n + 1 - i)
            part2 = 0
            for j in range(self.bv[k+1] - n + 1):
                if i + j % 2 == 0:
                    continue
                part2 += math.comb(self.bv[k+1] - n, j) * self.e_indirect(k + 2) ** j * (1 - self.e_indirect(k + 2)) ** (self.bv[k+1] - n - j)
            prob += part1 * part2
        return prob

    def e_logical_x(self):
        return self.e_indirect(0)

    def e_logical_z(self):
        b0 = self.bv[0]
        prob = 0

        def __e(_n):
            _prob = 0
            for i in range(_n + 1):
                p1 = binom_prob(self.epsilon, _n, i)
                p2 = 0
                for j in range(b0 - _n + 1):
                    if (i + j) % 2 == 0:
                        continue
                    p2 += math.comb(b0 - _n, j) * self.e_indirect(1) ** j * (1 - self.e_indirect(1)) ** (b0 - _n - j)
                _prob += p1 * p2
            return _prob

        for n in range(b0 + 1):
            # prob += math.comb(b0, n) * (self.r(1) / self.prob_mz(1)) ** (b0 - n) * (1 - (self.r(1) / self.prob_mz(1))) ** n * __e(n)
            prob += binom_prob(1 - self.prob_indirect_given_meas_success(1), n, b0) * __e(n)
        return prob

In [15]:
model = RgsModel(1, 5, [10, 5], 0.2, 0, 0.00002, 0.00002)

# model.prob_mz(0), model.prob_mz(1), model.prob_mz(2)
# model.e_indirect(0), model.e_indirect(1)
# model.e_indirect_single(0), model.e_indirect_single(1)

# model.e_indirect_single(1)
# model.e_indirect_given(1, 1)
# model.e_logical_x(), model.e_logical_z()
print(f'X error: {model.e_logical_x()}')
print(f'Z error: {model.e_logical_z()}')
# model.p_ph

n = 2
X error: 6.367759039526903e-06
Z error: 4.190202055653096e-53


In [16]:
bk = 7
ep = 0.1

def expand_method(bk, ep):
    prob = 0
    for l in range(1, bk + 2, 2):
        prob += math.comb(bk + 1, l) * ep ** l * (1 - ep) ** (bk + 1 - l)
    return prob

def shorten_method(bk, ep):
    return (1 - (1 - 2 * ep) ** (bk + 1)) / 2

expand_method(bk, ep), shorten_method(bk, ep)


(0.4161139200000001, 0.41611391999999997)

In [17]:
attenuation_distance = 10 / (np.log(10) * 0.2)
np.exp(-(2) / attenuation_distance)

0.9120108393559098

In [ ]:
# exhaustive calculation

epsilon = 0.00002
